# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [61]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [62]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [63]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [64]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [65]:
model_all = graphlab.linear_regression.create(sales, target='price', 
                                              features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., 
                                              l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.385163     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.413729     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [66]:
model_all_coefficients = model_all.get("coefficients")
model_all_coefficients.print_rows(num_rows=20)

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None |  274873.05595 |
|     bedrooms     |  None |      0.0      |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 8468.53108691 |
|   sqft_living    |  None | 24.4207209824 |
| sqft_living_sqrt |  None | 350.060553386 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 842.068034898 |
|    sqft_above    |  None | 20.0247224171 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION: According to this list of weights, which of the features have been chosen?*** 

In [67]:
for i in range(len(model_all_coefficients)):
    if model_all_coefficients[i]['value'] != 0 and model_all_coefficients[i]['name'] != '(intercept)':
        print model_all_coefficients[i]['name']

bathrooms
sqft_living
sqft_living_sqrt
grade
sqft_above


# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [68]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [69]:
import numpy as np

def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1
    features = ['constant'] + features # this is how you combine two lists
    
    features_sframe = data_sframe[features]
    feature_matrix = features_sframe.to_numpy()
    
    output_sarray = data_sframe[output]
    output_array = output_sarray.to_numpy()
    
    return(feature_matrix, output_array)

def predict_output(feature_matrix, weights):
    predictions = np.dot(feature_matrix, weights)
    return(predictions)

def get_rss(model, data, outcome):
    predictions = model.predict(data)
    residuals = outcome - predictions
    squared_residuals = residuals * residuals
    RSS = squared_residuals.sum()
    return(RSS)

my_output = 'price'

(feature_matrix, output) = get_numpy_data(training, all_features, my_output)
(test_feature_matrix, test_output) = get_numpy_data(testing, all_features, my_output)

min_rss = 0
min_rss_l1_penalty = 0

for l1_penalty in np.logspace(1, 7, num=13):
    l1_model = graphlab.linear_regression.create(training,
                                                 target='price',
                                                 features=all_features,
                                                 validation_set=None,
                                                 l2_penalty=0.,
                                                 l1_penalty = l1_penalty,
                                                 verbose=False)
    
    rss = get_rss(l1_model, validation, validation[my_output])
    
    if min_rss == 0:
        min_rss = rss
        min_rss_l1_penalty = l1_penalty
    else:
        if rss < min_rss:
            min_rss = rss
            min_rss_l1_penalty = l1_penalty
    
    print "l1_penalty=", l1_penalty, ": ", rss

print ""
print "min rss = ", min_rss
print "min rss l1 penalty = ", min_rss_l1_penalty

l1_penalty= 10.0 :  6.25766285142e+14
l1_penalty= 31.6227766017 :  6.25766285362e+14
l1_penalty= 100.0 :  6.25766286058e+14
l1_penalty= 316.227766017 :  6.25766288257e+14
l1_penalty= 1000.0 :  6.25766295212e+14
l1_penalty= 3162.27766017 :  6.25766317206e+14
l1_penalty= 10000.0 :  6.25766386761e+14
l1_penalty= 31622.7766017 :  6.25766606749e+14
l1_penalty= 100000.0 :  6.25767302792e+14
l1_penalty= 316227.766017 :  6.25769507644e+14
l1_penalty= 1000000.0 :  6.25776517727e+14
l1_penalty= 3162277.66017 :  6.25799062845e+14
l1_penalty= 10000000.0 :  6.25883719085e+14

min rss =  6.25766285142e+14
min rss l1 penalty =  10.0


*** QUIZ QUESTION: What was the best value for the `l1_penalty`?***

In [70]:
print min_rss_l1_penalty

10.0


***QUIZ QUESTION: What is the RSS on TEST data of the model with the best `l1_penalty`?***

In [72]:
best_l1_model = graphlab.linear_regression.create(training,
                                                 target='price',
                                                 features=all_features,
                                                 validation_set=None,
                                                 l2_penalty=0.,
                                                 l1_penalty = min_rss_l1_penalty,
                                                 verbose=False)

test_rss = get_rss(best_l1_model, testing, testing[my_output])

print test_rss

1.56983602382e+14


***QUIZ QUESTION: Also, using this value of L1 penalty, how many nonzero weights do you have?***

In [59]:
best_l1_model_coefficients = best_l1_model.get("coefficients")

num_nonzero_weights = 0

for i in range(len(best_l1_model_coefficients)):
    if best_l1_model_coefficients[i]['value'] != 0:
        print best_l1_model_coefficients[i]['name']
        num_nonzero_weights += 1

print ""
print "num nonzero weights = ", num_nonzero_weights

(intercept)
bedrooms
bedrooms_square
bathrooms
sqft_living
sqft_living_sqrt
sqft_lot
sqft_lot_sqrt
floors
floors_square
waterfront
view
condition
grade
sqft_above
sqft_basement
yr_built
yr_renovated

num nonzero weights =  18


# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [23]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [24]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [29]:
l1_penalty_min = 0
l1_penalty_max = l1_penalty_values[len(l1_penalty_values)-1]

for l1_penalty in l1_penalty_values:
    new_l1_model = graphlab.linear_regression.create(training,
                                                     target='price',
                                                     features=all_features,
                                                     validation_set=None,
                                                     l2_penalty=0.,
                                                     l1_penalty = l1_penalty,
                                                     verbose=False)
    
    new_l1_model_weights = new_l1_model['coefficients']['value']
    num_nonzero_weights = new_l1_model_weights.nnz()
    
    if l1_penalty > l1_penalty_min and num_nonzero_weights > max_nonzeros:
        l1_penalty_min = l1_penalty
    
    if l1_penalty < l1_penalty_max and num_nonzero_weights < max_nonzeros:
        l1_penalty_max = l1_penalty
    
    print "l1_penalty = ", l1_penalty, ": non-zero weights = ", num_nonzero_weights

l1_penalty =  100000000.0 : non-zero weights =  18
l1_penalty =  127427498.57 : non-zero weights =  18
l1_penalty =  162377673.919 : non-zero weights =  18
l1_penalty =  206913808.111 : non-zero weights =  18
l1_penalty =  263665089.873 : non-zero weights =  17
l1_penalty =  335981828.628 : non-zero weights =  17
l1_penalty =  428133239.872 : non-zero weights =  17
l1_penalty =  545559478.117 : non-zero weights =  17
l1_penalty =  695192796.178 : non-zero weights =  17
l1_penalty =  885866790.41 : non-zero weights =  16
l1_penalty =  1128837891.68 : non-zero weights =  15
l1_penalty =  1438449888.29 : non-zero weights =  15
l1_penalty =  1832980710.83 : non-zero weights =  13
l1_penalty =  2335721469.09 : non-zero weights =  12
l1_penalty =  2976351441.63 : non-zero weights =  10
l1_penalty =  3792690190.73 : non-zero weights =  6
l1_penalty =  4832930238.57 : non-zero weights =  5
l1_penalty =  6158482110.66 : non-zero weights =  3
l1_penalty =  7847599703.51 : non-zero weights =  1
l

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

***QUIZ QUESTIONS: What values did you find for `l1_penalty_min` and`l1_penalty_max`?*** 

In [56]:
print "l1_penalty_min =", ("{0:.0f}".format(round(l1_penalty_min, 0)))
print "l1_penalty_max =", ("{0:.0f}".format(round(l1_penalty_max, 0)))

l1_penalty_min =  2976351442
l1_penalty_max =  3792690191


## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [33]:
narrow_l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [37]:
min_rss = 0
min_rss_l1_penalty = 0

rss_and_numzeros_per_l1_penalty = {}

for l1_penalty in narrow_l1_penalty_values:
    narrow_l1_model = graphlab.linear_regression.create(training,
                                                        target=my_output,
                                                        features=all_features,
                                                        validation_set=None,
                                                        l2_penalty=0.,
                                                        l1_penalty=l1_penalty,
                                                        verbose=False)
    
    rss = get_rss(narrow_l1_model, validation, validation[my_output])
    
    narrow_l1_model_weights = narrow_l1_model['coefficients']['value']
    num_nonzero_weights = narrow_l1_model_weights.nnz()
    
    rss_and_numzeros = [rss, num_nonzero_weights]
    rss_and_numzeros_per_l1_penalty[l1_penalty] = rss_and_numzeros
    
    if min_rss == 0:
        min_rss = rss
        min_rss_l1_penalty = l1_penalty
    else:
        if rss < min_rss:
            min_rss = rss
            min_rss_l1_penalty = l1_penalty
    
    print "l1_penalty =", l1_penalty, ": rss = ", rss, "; nonzeros = ", num_nonzero_weights

print ""
print "min rss = ", min_rss
print "min rss l1 penalty = ", min_rss_l1_penalty

l1_penalty = 2976351441.63 : rss =  9.66925692362e+14 ; nonzeros =  10
l1_penalty = 3019316638.95 : rss =  9.74019450085e+14 ; nonzeros =  10
l1_penalty = 3062281836.27 : rss =  9.81188367942e+14 ; nonzeros =  10
l1_penalty = 3105247033.59 : rss =  9.89328342459e+14 ; nonzeros =  10
l1_penalty = 3148212230.92 : rss =  9.98783211266e+14 ; nonzeros =  10
l1_penalty = 3191177428.24 : rss =  1.00847716702e+15 ; nonzeros =  10
l1_penalty = 3234142625.56 : rss =  1.01829878055e+15 ; nonzeros =  10
l1_penalty = 3277107822.88 : rss =  1.02824799221e+15 ; nonzeros =  10
l1_penalty = 3320073020.2 : rss =  1.03461690923e+15 ; nonzeros =  8
l1_penalty = 3363038217.52 : rss =  1.03855473594e+15 ; nonzeros =  8
l1_penalty = 3406003414.84 : rss =  1.04323723787e+15 ; nonzeros =  8
l1_penalty = 3448968612.16 : rss =  1.04693748875e+15 ; nonzeros =  7
l1_penalty = 3491933809.48 : rss =  1.05114762561e+15 ; nonzeros =  7
l1_penalty = 3534899006.81 : rss =  1.05599273534e+15 ; nonzeros =  7
l1_penalty = 

***QUIZ QUESTION: What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?***

In [41]:
max_nonzero_l1_penalty = 0

for l1_penalty in rss_and_numzeros_per_l1_penalty:
    if rss_and_numzeros_per_l1_penalty[l1_penalty][1] == max_nonzeros:
        if max_nonzero_l1_penalty == 0:
            max_nonzero_l1_penalty = l1_penalty
        elif rss_and_numzeros_per_l1_penalty[l1_penalty] < rss_and_numzeros_per_l1_penalty[max_nonzero_l1_penalty]:
            max_nonzero_l1_penalty = l1_penalty

In [58]:
print ("{0:.0f}".format(round(max_nonzero_l1_penalty, 0)))

3448968612


***QUIZ QUESTION: What features in this model have non-zero coefficients?***

In [60]:
best_l1_penalty_model = graphlab.linear_regression.create(training,
                                                          target=my_output,
                                                          features=all_features,
                                                          l2_penalty=0.,
                                                          l1_penalty=max_nonzero_l1_penalty,
                                                          validation_set=None,
                                                          verbose=False)

best_l1_penalty_model_weights = best_l1_penalty_model.get('coefficients')
print best_l1_penalty_model_weights.print_rows(num_rows=20)

for coefficient in best_l1_penalty_model_weights:
    if coefficient['value'] != 0:
        print coefficient['name']    

+------------------+-------+---------------+
|       name       | index |     value     |
+------------------+-------+---------------+
|   (intercept)    |  None | 222253.192544 |
|     bedrooms     |  None | 661.722717782 |
| bedrooms_square  |  None |      0.0      |
|    bathrooms     |  None | 15873.9572593 |
|   sqft_living    |  None | 32.4102214513 |
| sqft_living_sqrt |  None | 690.114773313 |
|     sqft_lot     |  None |      0.0      |
|  sqft_lot_sqrt   |  None |      0.0      |
|      floors      |  None |      0.0      |
|  floors_square   |  None |      0.0      |
|    waterfront    |  None |      0.0      |
|       view       |  None |      0.0      |
|    condition     |  None |      0.0      |
|      grade       |  None | 2899.42026975 |
|    sqft_above    |  None | 30.0115753022 |
|  sqft_basement   |  None |      0.0      |
|     yr_built     |  None |      0.0      |
|   yr_renovated   |  None |      0.0      |
+------------------+-------+---------------+
[18 rows x